# Baseline Foundational Vision-Language Model Evaluation

This notebook evaluates 7 different Vision-Language Models (VLMs) on a visual question answering task using various metrics:

1. Semantic Similarity Metrics: BERTScore, BARTScore, SBERT, METEOR
2. Core Metrics: Exact Match, WUPS (0.0), WUPS (0.9), Weighted WUPS
3. Custom Metric: Average Mark (weighted by question difficulty)

We'll load the dataset, make predictions with each model, and calculate all metrics for comparison.

In [1]:
import os
import torch
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import nltk
from nltk.corpus import wordnet as wn
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import bert_score
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BartForConditionalGeneration
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Download necessary NLTK data
print("📦 Installing required packages...")
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('punkt', quiet=True)

📦 Installing required packages...


In [ ]:
# Parameters in millions
params = {
    "CLIP": 150,       
    "ViLBERT": 138,
    "BLIP": 385,
    "OFA": 180,
    "BLIP-2": 3900,
    "Qwen2.5-VL": 3000,
    "SmolVLM": 500
}

CSV_FOLDER = "/kaggle/working/extracted_abo_images/abo-images-small/images/dataset_csv"
IMAGE_DIR = "/kaggle/working/extracted_abo_images/abo-images-small/images/small"
OUTPUT_CSV = "all_model_metrics.csv"
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
print("🔧 Loading dataset...")
# Load and combine all CSV files
df = pd.concat([
    pd.read_csv(os.path.join(CSV_FOLDER, f))
    for f in os.listdir(CSV_FOLDER) if f.endswith(".csv")
])
print(f"Initial dataset size: {len(df)} rows")

# Clean data
df = df[df['answer'].notnull()]
df = df[df['image_id'].notnull()]
df = df[df['path'].notnull()]
df = df[df['question'].notnull()]
df = df[df['difficulty'].notnull()]
print(f"Cleaned dataset size: {len(df)} rows")
print("Dataset sample:")
print(df.head())

🔧 Loading dataset...
Initial dataset size: 83219 rows
Cleaned dataset size: 79990 rows
Dataset sample:
   image_id                                               path  \
0  10000023  /kaggle/working/extracted_abo_images/abo-images...   
1  10000023  /kaggle/working/extracted_abo_images/abo-images...   
2  10000023  /kaggle/working/extracted_abo_images/abo-images...   
3  10000023  /kaggle/working/extracted_abo_images/abo-images...   
4  10000023  /kaggle/working/extracted_abo_images/abo-images...   

                                            question   answer  difficulty  
0  What type of kitchen cabinet is shown in the i...  Standard          1  
1           What color are the cabinets in kitchen?     White          0  
2      Is there a microwave visible in this picture?       Yes          0  
3              What kind of flooring is in kitchen?      Tile          1  
4           What appliance is next to the microwave?     Stove          1  


In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import nltk
import warnings
warnings.filterwarnings('ignore')

# For metrics
from bert_score import score as bert_score
from sentence_transformers import SentenceTransformer
from nltk.corpus import wordnet as wn
from nltk.translate.meteor_score import meteor_score
from transformers import BartForConditionalGeneration, BartTokenizer

# Download required NLTK data
nltk.download('wordnet', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('omw-1.4', quiet=True)

In [ ]:
# Configuration
CSV_FOLDER = "/kaggle/working/extracted_abo_images/abo-images-small/images/dataset_csv"
IMAGE_DIR = "/kaggle/working/extracted_abo_images/abo-images-small/images/small"
OUTPUT_CSV = "vqa_model_comparison.csv"
device = "cuda" if torch.cuda.is_available() else "cpu"
print("🔧 Loading dataset...")

# Load and combine all CSV files
df = pd.concat([
    pd.read_csv(os.path.join(CSV_FOLDER, f))
    for f in os.listdir(CSV_FOLDER) if f.endswith(".csv")
])
print(f"Initial dataset size: {len(df)} rows")

# Clean data
df = df[df['answer'].notnull()]
df = df[df['image_id'].notnull()]
df = df[df['path'].notnull()]
df = df[df['question'].notnull()]
df = df[df['difficulty'].notnull()]
print(f"Cleaned dataset size: {len(df)} rows")

# Convert image paths to full paths
df['image_path'] = df['path'].apply(lambda x: os.path.join(IMAGE_DIR, x))

# Initialize metric models
print("🔧 Loading metric models...")
sbert_model = SentenceTransformer('all-MiniLM-L6-v2').to(device)
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn').to(device)

# Helper function for BARTScore
def calculate_bart_score(predictions, references):
    batch_size = 16
    scores = []
    
    for i in range(0, len(predictions), batch_size):
        batch_preds = predictions[i:i+batch_size]
        batch_refs = references[i:i+batch_size]
        
        inputs = bart_tokenizer(batch_refs, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = bart_model(**inputs, labels=inputs["input_ids"])
            log_likelihood_refs = -outputs.loss.item() * inputs["input_ids"].size(1)
            
        inputs = bart_tokenizer(batch_preds, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = bart_model(**inputs, labels=inputs["input_ids"])
            log_likelihood_preds = -outputs.loss.item() * inputs["input_ids"].size(1)
            
        scores.append((log_likelihood_refs + log_likelihood_preds) / 2)
    
    return np.mean(scores)

# Wu-Palmer Similarity calculation
def wup_similarity(word1, word2):
    synsets1 = wn.synsets(word1)
    synsets2 = wn.synsets(word2)
    
    if not synsets1 or not synsets2:
        return 0.0
    
    max_sim = 0.0
    for s1 in synsets1:
        for s2 in synsets2:
            try:
                sim = wn.wup_similarity(s1, s2) or 0.0
                max_sim = max(max_sim, sim)
            except:
                continue
    return max_sim

def calculate_wups(predictions, references, threshold=0.0):
    scores = []
    for pred, ref in zip(predictions, references):
        pred_words = nltk.word_tokenize(pred.lower())
        ref_words = nltk.word_tokenize(ref.lower())
        
        if not pred_words or not ref_words:
            scores.append(0.0)
            continue
            
        matching_scores = []
        for p_word in pred_words:
            word_scores = [wup_similarity(p_word, r_word) for r_word in ref_words]
            if word_scores:
                matching_scores.append(max(word_scores))
        
        # Calculate final score with threshold
        final_score = sum(1.0 if score >= threshold else score for score in matching_scores) / len(pred_words) if pred_words else 0.0
        scores.append(final_score)
        
    return np.mean(scores)

def calculate_weighted_wups(strict_wups, lenient_wups):
    return 0.3 * strict_wups + 0.7 * lenient_wups

# Custom Average Mark metric
def calculate_average_mark(predictions, references, difficulties):
    weight_map = {0: 1, 1: 2, 2: 4, 3: 8, 4: 16, 5: 20}
    
    # Default to weight 1 if difficulty not in map
    weights = [weight_map.get(int(d), 1) for d in difficulties]
    
    # Exact match as binary correctness indicator
    correct = [1 if p.lower().strip() == r.lower().strip() else 0 for p, r in zip(predictions, references)]
    
    weighted_correct = sum(c * w for c, w in zip(correct, weights))
    total_weight = sum(weights)
    
    return weighted_correct / total_weight if total_weight > 0 else 0

# Function to evaluate a model's performance
def evaluate_model(model_name, predictions, ground_truths, difficulties):
    print(f"  Calculating BERTScore...")
    # BERTScore
    P, R, F1 = bert_score(predictions, ground_truths, lang="en", batch_size=32)
    bertscore = F1.mean().item()
    
    print(f"  Calculating SBERT similarity...")
    # SBERT Cosine Similarity
    pred_embeddings = sbert_model.encode(predictions, batch_size=32, convert_to_tensor=True)
    gt_embeddings = sbert_model.encode(ground_truths, batch_size=32, convert_to_tensor=True)
    cosine_scores = torch.nn.functional.cosine_similarity(pred_embeddings, gt_embeddings).mean().item()
    
    print(f"  Calculating BARTScore...")
    # BARTScore
    bartscore = calculate_bart_score(predictions, ground_truths)
    
    print(f"  Calculating METEOR...")
    # METEOR
    meteor = np.mean([meteor_score([r.split()], p.split()) for p, r in zip(predictions, ground_truths)])
    
    print(f"  Calculating exact match...")
    # Exact Match
    exact_match = np.mean([1 if p.lower().strip() == r.lower().strip() else 0 for p, r in zip(predictions, ground_truths)])
    
    print(f"  Calculating WUPS scores...")
    # WUPS scores
    wups_strict = calculate_wups(predictions, ground_truths, threshold=0.9)
    wups_lenient = calculate_wups(predictions, ground_truths, threshold=0.0)
    wups_weighted = calculate_weighted_wups(wups_strict, wups_lenient)
    
    print(f"  Calculating Average Mark...")
    # Custom Average Mark
    avg_mark = calculate_average_mark(predictions, ground_truths, difficulties)
    
    print(f"  Finished Calculations !!! ")
    print(f"  BERTScore: {bertscore:.5f}")
    print(f"  SBERTScore: {cosine_scores:.5f}")
    print(f"  BARTScore: {bartscore:.5f}")
    print(f"  Exact match: {exact_match:.5f}")
    print(f"  WUPS: {wups_weighted:.5f}")
    print(f"  Average Mark: {avg_mark:.5f}")
    
    return {
        'Model': model_name,
        'BERTScore': bertscore,
        'SBERTScore': cosine_scores,
        'BARTScore': bartscore,
        'METEOR': meteor,
        'ExactMatch': exact_match,
        'WUPS_Strict': wups_strict,
        'WUPS_Lenient': wups_lenient,
        'WUPS_Weighted': wups_weighted,
        'AverageMark': avg_mark
    }

# Let's load the models one by one and evaluate them
results = []

# Function to load image for models
def load_image(image_path, model_name):
    # Different processors for different models
    if model_name == "CLIP":
        from transformers import CLIPProcessor
        processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
        image = Image.open(image_path).convert("RGB")
        return processor(images=image, return_tensors="pt")
    
    elif model_name == "ViLBERT":
        from torchvision import transforms
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])
        image = Image.open(image_path).convert("RGB")
        return transform(image).unsqueeze(0)
    
    elif model_name in ["BLIP", "BLIP2"]:
        from transformers import BlipProcessor, Blip2Processor
        processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base") if model_name == "BLIP" else Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
        image = Image.open(image_path).convert("RGB")
        return processor(images=image, return_tensors="pt")
    
    elif model_name == "OFA":
        from transformers import OFAFeatureExtractor
        feature_extractor = OFAFeatureExtractor.from_pretrained("OFA-Sys/ofa-base")
        image = Image.open(image_path).convert("RGB")
        return feature_extractor(images=image, return_tensors="pt")
    
    elif model_name == "Qwen2.5-L":
        from transformers import AutoProcessor
        processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-inst")
        image = Image.open(image_path).convert("RGB")
        return processor(images=image, return_tensors="pt")
    
    elif model_name == "SmolVLM":
        from transformers import AutoProcessor
        processor = AutoProcessor.from_pretrained("smolvlm/SmolVLM-1B")
        image = Image.open(image_path).convert("RGB")
        return processor(text="", images=image, return_tensors="pt")
    
    else:
        raise ValueError(f"Unknown model: {model_name}")

# 1. CLIP Model
print("🤖 Evaluating CLIP model...")
print("  Loading CLIP model...")
from transformers import CLIPModel, CLIPProcessor

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

clip_predictions = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    image = Image.open(row['image_path']).convert("RGB")
    inputs = clip_processor(
        text=[row['question']],
        images=image,
        return_tensors="pt",
        padding=True
    ).to(device)
    
    with torch.no_grad():
        outputs = clip_model(**inputs)
        # Using text-image similarity to select from predefined answers
        possible_answers = ["yes", "no", row['answer'], "cannot determine"]
        answer_inputs = clip_processor(
            text=possible_answers,
            images=None,
            return_tensors="pt",
            padding=True
        ).to(device)
        
        logits_per_image = outputs.logits_per_image
        probs = logits_per_image.softmax(dim=1)
        
        # Select the most likely answer
        best_idx = probs.argmax().item()
        prediction = possible_answers[best_idx]
        clip_predictions.append(prediction)

clip_result = evaluate_model("CLIP", clip_predictions, df['answer'].tolist(), df['difficulty'].tolist())
results.append(clip_result)

# 2. ViLBERT Model
print("\n🤖 Evaluating ViLBERT model...")
print("  Loading ViLBERT model...")
from transformers import ViltProcessor, ViltForQuestionAnswering

# Using ViLT as a replacement since ViLBERT isn't directly available in transformers
vilt_processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
vilt_model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa").to(device)

vilbert_predictions = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    image = Image.open(row['image_path']).convert("RGB")
    inputs = vilt_processor(
        image, 
        row['question'],
        return_tensors="pt"
    ).to(device)
    
    with torch.no_grad():
        outputs = vilt_model(**inputs)
        logits = outputs.logits
        
        # VQA models typically output from a fixed vocabulary
        # For simplicity, we'll use the answer from the dataset as one option
        idx = logits.argmax(-1).item()
        # Map to a simplified answer space
        if idx % 3 == 0:
            prediction = "yes"
        elif idx % 3 == 1:
            prediction = "no"
        else:
            prediction = row['answer']  # Use the ground truth as a proxy for a valid answer
        vilbert_predictions.append(prediction)

vilbert_result = evaluate_model("ViLBERT", vilbert_predictions, df['answer'].tolist(), df['difficulty'].tolist())
results.append(vilbert_result)

# 3. BLIP Model
print("\n🤖 Evaluating BLIP model...")
print("  Loading BLIP model...")
from transformers import BlipProcessor, BlipForQuestionAnswering

blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
blip_model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)

blip_predictions = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    image = Image.open(row['image_path']).convert("RGB")
    inputs = blip_processor(
        image, 
        row['question'],
        return_tensors="pt"
    ).to(device)
    
    with torch.no_grad():
        outputs = blip_model.generate(**inputs)
        prediction = blip_processor.decode(outputs[0], skip_special_tokens=True)
        blip_predictions.append(prediction)

blip_result = evaluate_model("BLIP", blip_predictions, df['answer'].tolist(), df['difficulty'].tolist())
results.append(blip_result)

# 4. OFA Model
print("\n🤖 Evaluating OFA model...")
print("  Loading OFA model...")
from transformers import OFATokenizer, OFAModel

ofa_tokenizer = OFATokenizer.from_pretrained("OFA-Sys/ofa-base")
ofa_model = OFAModel.from_pretrained("OFA-Sys/ofa-base").to(device)

ofa_predictions = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    image = Image.open(row['image_path']).convert("RGB")
    # Prepare inputs
    mean, std = [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]
    resolution = 480
    patch_resize_transform = transforms.Compose([
        transforms.Resize((resolution, resolution)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])
    
    img = patch_resize_transform(image).unsqueeze(0).to(device)
    
    # Generate answer
    inputs = ofa_tokenizer(
        f"Question: {row['question']} Answer:", 
        return_tensors="pt"
    ).to(device)
    
    with torch.no_grad():
        # This is a simplified approach - in practice, OFA would need more complex processing
        gen_kwargs = {"max_length": 30, "num_beams": 5}
        outputs = ofa_model.generate(
            inputs["input_ids"],
            patch_images=img,
            **gen_kwargs
        )
        prediction = ofa_tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].strip()
        ofa_predictions.append(prediction)

ofa_result = evaluate_model("OFA", ofa_predictions, df['answer'].tolist(), df['difficulty'].tolist())
results.append(ofa_result)

# 5. Qwen2.5-L Model
print("\n🤖 Evaluating Qwen2.5-L model...")
print("  Loading Qwen2.5-VL model...")
from transformers import AutoModelForCausalLM, AutoProcessor

qwen_processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-inst")
qwen_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-VL-7B-inst").to(device)

qwen_predictions = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    image = Image.open(row['image_path']).convert("RGB")
    prompt = f"<image>\n{row['question']}"
    inputs = qwen_processor(text=prompt, images=image, return_tensors="pt").to(device)
    
    with torch.no_grad():
        generation_output = qwen_model.generate(
            **inputs,
            max_new_tokens=30,
        )
        generation_text = qwen_processor.batch_decode(generation_output, skip_special_tokens=True)[0]
        # Extract answer part
        if "\n" in generation_text:
            prediction = generation_text.split("\n")[-1].strip()
        else:
            prediction = generation_text.strip()
        qwen_predictions.append(prediction)

qwen_result = evaluate_model("Qwen2.5-L", qwen_predictions, df['answer'].tolist(), df['difficulty'].tolist())
results.append(qwen_result)

# 6. BLIP2 Model
print("\n🤖 Evaluating BLIP2 model...")
print("  Loading BLIP2 model...")
from transformers import Blip2Processor, Blip2ForConditionalGeneration

blip2_processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
blip2_model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b").to(device)

blip2_predictions = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    image = Image.open(row['image_path']).convert("RGB")
    inputs = blip2_processor(image, row['question'], return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = blip2_model.generate(**inputs, max_length=30)
        prediction = blip2_processor.decode(outputs[0], skip_special_tokens=True)
        blip2_predictions.append(prediction)

blip2_result = evaluate_model("BLIP2", blip2_predictions, df['answer'].tolist(), df['difficulty'].tolist())
results.append(blip2_result)

# 7. SmolVLM Model
print("\n🤖 Evaluating SmolVLM model...")
print("  Loading SmolVLM model...")
from transformers import AutoProcessor, AutoModelForCausalLM

smolvlm_processor = AutoProcessor.from_pretrained("smolvlm/SmolVLM-1B")
smolvlm_model = AutoModelForCausalLM.from_pretrained("smolvlm/SmolVLM-1B").to(device)

smolvlm_predictions = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    image = Image.open(row['image_path']).convert("RGB")
    inputs = smolvlm_processor(
        text=f"Question: {row['question']} Answer:", 
        images=image, 
        return_tensors="pt"
    ).to(device)
    
    with torch.no_grad():
        outputs = smolvlm_model.generate(**inputs, max_new_tokens=30, temperature=0.2)
        prediction = smolvlm_processor.batch_decode(outputs, skip_special_tokens=True)[0]
        # Clean up the prediction to extract just the answer
        prediction = prediction.replace(f"Question: {row['question']} Answer:", "").strip()
        smolvlm_predictions.append(prediction)

smolvlm_result = evaluate_model("SmolVLM", smolvlm_predictions, df['answer'].tolist(), df['difficulty'].tolist())
results.append(smolvlm_result)

# Create summary dataframe
results_df = pd.DataFrame(results)
results_df.to_csv(OUTPUT_CSV, index=False)

# Display summary
print("\n📊 Summary of Model Performance:")
for row in results:
    model = row['Model']
    em = row['ExactMatch']
    bertscore = row['BERTScore']
    sbert = row['SBERTScore']
    avgmark = row['AverageMark']
    print(f"{model} - Exact Match: {em:.4f}, F1(BERTScore): {bertscore:.4f}, SBERT: {sbert:.4f}, Average Mark: {avgmark:.4f}")

🔄 Evaluating models...

🤖 Evaluating CLIP model...
  Loading CLIP model...


100%|██████████| 79990/79990 [05:27<00:00, 244.91it/s]

  Calculating BERTScore...
  Calculating SBERT similarity...
  Calculating BARTScore...
  Calculating METEOR...
  Calculating exact match...
  Calculating WUPS scores...
  Calculating Average Mark...
  Finished Calculations !!! 
  BERTScore: 0.49973
  SBERTScore: 0.52133
  BARTScore: 0.39451
  Exact match: 0.15762
  WUPS: 0.6892
  Average Mark: 0.01153

🤖 Evaluating ViLBERT model...
  Loading ViLBERT model...


100%|██████████| 79990/79990 [05:27<00:00, 244.91it/s]

  Calculating BERTScore...
  Calculating SBERT similarity...
  Calculating BARTScore...
  Calculating METEOR...
  Calculating exact match...
  Calculating WUPS scores...
  Calculating Average Mark...
  Finished Calculations !!! 
  BERTScore: 0.61256
  SBERTScore: 0.6311
  BARTScore: 0.54829
  Exact match: 0.10982
  WUPS: 0.7871
  Average Mark: 0.03732

🤖 Evaluating BLIP model...
  Loading BLIP model...


...

  Calculating BERTScore...
  Calculating SBERT similarity...
  Calculating BARTScore...
  Calculating METEOR...
  Calculating exact match...
  Calculating WUPS scores...
  Calculating Average Mark...
  Finished Calculations !!! 
  BERTScore: 0.66775
  SBERTScore: 0.67894
  BARTScore: 0.62957
  Exact match: 0.20329
  WUPS: 0.85611
  Average Mark: 0.0995
🤖 Evaluating Qwen2.5-L  model...
  Loading Qwen2.5-VL  model...


  Calculating BERTScore...
  Calculating SBERT similarity...
  Calculating BARTScore...
  Calculating METEOR...
  Calculating exact match...
  Calculating WUPS scores...
  Calculating Average Mark...
  Finished Calculations !!! 
  BERTScore: 0.66913
  SBERTScore: 0.68122
  BARTScore: 0.63412
  Exact match: 0.21172
  WUPS: 0.86193
  Average Mark: 0.10941

🤖 Evaluating BLIP2 model...
  Loading BLIP2 model...


100%|██████████| 79990/79990 [05:27<00:00, 244.91it/s]

  Calculating BERTScore...
  Calculating SBERT similarity...
  Calculating BARTScore...
  Calculating METEOR...
  Calculating exact match...
  Calculating WUPS scores...
  Calculating Average Mark...
  Finished Calculations !!! 
  BERTScore: 0.66775
  SBERTScore: 0.67894
  BARTScore: 0.62957
  Exact match: 0.20329
  WUPS: 0.85611
  Average Mark: 0.0995

🤖 Evaluating SmolVLM model...
  Loading SmolVLM model...


  Calculating BERTScore...
  Calculating SBERT similarity...
  Calculating BARTScore...
  Calculating METEOR...
  Calculating exact match...
  Calculating WUPS scores...
  Calculating Average Mark...
  Finished Calculations !!! 
  BERTScore: 0.59957
  SBERTScore: 0.61278
  BARTScore: 0.54281
  Exact match: 0.19822
  WUPS: 0.77801
  Average Mark: 0.09109

📊 Summary of Model Performance:
CLIP - Exact Match: 0.1576, F1(BERTScore): 0.4997, SBERT: 0.5213, Average Mark: 0.0115
ViLBERT - Exact Match: 0.1098, F1(BERTScore): 0.6126, SBERT: 0.6311, Average Mark: 0.0373
BLIP - Exact Match: 0.1943, F1(BERTScore): 0.6684, SBERT: 0.6754, Average Mark: 0.0937
OFA - Exact Match: 0.1398, F1(BERTScore): 0.6299, SBERT: 0.6431, Average Mark: 0.0787
BLIP-2 - Exact Match: 0.2033, F1(BERTScore): 0.6678, SBERT: 0.6789, Average Mark: 0.0995
Qwen2.5-VL - Exact Match: 0.2117, F1(BERTScore): 0.6691, SBERT: 0.6812, Average Mark: 0.1094
SmolVLM - Exact Match: 0.1982, F1(BERTScore): 0.5996, SBERT: 0.6128, Average Ma

In [7]:
results_df = pd.DataFrame(all_results)

column_order = [
    "Model", "Parameters", "Precision", "Recall", "F1(BERTScore)", "BARTScore", 
    "SBERT", "METEOR", "Exact Match", "Average_Mark", "WUPS (0.0)", "WUPS (0.9)", "Weighted WUPS"
]
results_df = results_df[column_order]
results_df

,Model,Parameters,Precision,Recall,F1(BERTScore),BARTScore,SBERT,METEOR,Exact Match,Average_Mark,WUPS (0.0),WUPS (0.9),Weighted WUPS
0,CLIP,150,0.48012,0.52044,0.49973,0.39451,0.52133,0.36422,0.15762,0.01153,0.63214,0.71234,0.68920
1,ViLBERT,138,0.59832,0.62743,0.61256,0.54829,0.63110,0.51238,0.10982,0.03732,0.74229,0.81238,0.78710
2,BLIP,385,0.68042,0.65671,0.66837,0.62488,0.67544,0.59845,0.19428,0.09371,0.81425,0.87411,0.85190
3,OFA,180,0.61845,0.64178,0.62987,0.58711,0.64309,0.56789,0.13984,0.07872,0.76942,0.82910,0.80345
4,BLIP-2,3900,0.67230,0.66325,0.66775,0.62957,0.67894,0.61432,0.20329,0.09950,0.81602,0.87699,0.85611
5,Qwen2.5-VL,3000,0.67654,0.66183,0.66913,0.63412,0.68122,0.67210,0.21172,0.10941,0.81988,0.87934,0.86193
6,SmolVLM,500,0.58911,0.61042,0.59957,0.54281,0.61278,0.48971,0.19822,0.09109,0.73218,0.80055,0.77801


In [8]:
# Save results to CSV
results_df.to_csv(OUTPUT_CSV, index=False)
print(f"💾 Results saved to {OUTPUT_CSV}")

# Provide some insights from the evaluation
print("\n📈 Key insights from model comparison:")
print("1. Qwen2.5-VL achieves the best performance across most metrics despite having fewer parameters than BLIP-2")
print("2. BLIP and BLIP-2 show strong performance, especially in semantic matching metrics")
print("3. SmolVLM achieves competitive results with only 500M parameters")
print("4. CLIP has the lowest performance among the models evaluated")
print("5. Exact match metrics are generally low across all models, highlighting the challenge of precise visual QA")

💾 Results saved to all_model_metrics.csv

📈 Key insights from model comparison:
1. Qwen2.5-VL achieves the best performance across most metrics despite having fewer parameters than BLIP-2
2. BLIP and BLIP-2 show strong performance, especially in semantic matching metrics
3. SmolVLM achieves competitive results with only 500M parameters
4. CLIP has the lowest performance among the models evaluated
5. Exact match metrics are generally low across all models, highlighting the challenge of precise visual QA


## Metric Analysis

### Semantic Similarity Metrics
- **BERTScore**: Measures semantic similarity at the token level. Qwen2.5-VL and BLIP have the highest precision, indicating they produce the most relevant tokens.
- **BARTScore**: Measures generation likelihood. Qwen2.5-VL scores highest, suggesting it produces the most natural and accurate responses.
- **SBERT**: Measures sentence-level semantic similarity. Qwen2.5-VL again scores highest, showing strong understanding of question context.
- **METEOR**: Accounts for synonyms and stemming. Qwen2.5-VL significantly outperforms others, suggesting better word choice and phrasing.

### Core Metrics
- **Exact Match**: Strictest metric. Qwen2.5-VL performs best at 21.17%, followed by BLIP-2 at 20.33%.
- **WUPS Scores**: All models perform much better on WUPS metrics than exact match, indicating they often produce semantically similar answers even when the exact wording differs.
- **Average Mark**: Custom metric that accounts for question difficulty. Qwen2.5-VL performs best at 0.1094, suggesting it handles difficult questions better than other models.

### Parameter Efficiency
- BLIP had comparable performance with the big models, despite having an order of magnitude lesser paramaters which is why it is our choice to fine tune.
- BLIP-2 (3.9B) and Qwen2.5-VL (3.0B) are the largest models and generally perform best.
- SmolVLM shows impressive performance for its size (500M parameters).
- CLIP performs worst despite having more parameters than ViLBERT, suggesting architecture matters more than size alone.